In [1]:
import os
import numpy as np
import torch
import torch.distributed as dist

os.makedirs('/content/drive/MyDrive/yolact/results/images/', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolact/results/trt_images/', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolact/weights/', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolact/trt_files/', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolact/tensorboard_log/', exist_ok=True)

COLORS = np.array([[0, 0, 0], [244, 67, 54], [233, 30, 99], [156, 39, 176], [103, 58, 183], [100, 30, 60],
                   [63, 81, 181], [33, 150, 243], [3, 169, 244], [0, 188, 212], [20, 55, 200],
                   [0, 150, 136], [76, 175, 80], [139, 195, 74], [205, 220, 57], [70, 25, 100],
                   [255, 235, 59], [255, 193, 7], [255, 152, 0], [255, 87, 34], [90, 155, 50],
                   [121, 85, 72], [158, 158, 158], [96, 125, 139], [15, 67, 34], [98, 55, 20],
                   [21, 82, 172], [58, 128, 255], [196, 125, 39], [75, 27, 134], [90, 125, 120],
                   [121, 82, 7], [158, 58, 8], [96, 25, 9], [115, 7, 234], [8, 155, 220],
                   [221, 25, 72], [188, 58, 158], [56, 175, 19], [215, 67, 64], [198, 75, 20],
                   [62, 185, 22], [108, 70, 58], [160, 225, 39], [95, 60, 144], [78, 155, 120],
                   [101, 25, 142], [48, 198, 28], [96, 225, 200], [150, 167, 134], [18, 185, 90],
                   [21, 145, 172], [98, 68, 78], [196, 105, 19], [215, 67, 84], [130, 115, 170],
                   [255, 0, 255], [255, 255, 0], [196, 185, 10], [95, 167, 234], [18, 25, 190],
                   [0, 255, 255], [255, 0, 0], [0, 255, 0], [0, 0, 255], [155, 0, 0],
                   [0, 155, 0], [0, 0, 155], [46, 22, 130], [255, 0, 155], [155, 0, 255],
                   [255, 155, 0], [155, 255, 0], [0, 155, 255], [0, 255, 155], [18, 5, 40],
                   [120, 120, 255], [255, 58, 30], [60, 45, 60], [75, 27, 244], [128, 25, 70]], dtype='uint8')

COCO_LABEL_MAP = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8,
                  9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
                  18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24,
                  27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32,
                  37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40,
                  46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48,
                  54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56,
                  62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64,
                  74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
                  82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}

CUSTOM_CLASSES = ('cells','kernel')
# CUSTOM_CLASSES = ('dog', 'person', 'bear', 'sheep')

norm_mean = np.array([103.94, 116.78, 123.68], dtype=np.float32)
norm_std = np.array([57.38, 57.12, 58.40], dtype=np.float32)

class res101_coco:
    def __init__(self, args):
        self.mode = args.mode
        self.cuda = args.cuda
        self.gpu_id = 0
        assert args.img_size % 32 == 0, f'Img_size must be divisible by 32, got {args.img_size}.'
        self.img_size = args.img_size
        self.class_names = CUSTOM_CLASSES
        self.num_classes = len(CUSTOM_CLASSES) + 1
        self.continuous_id = {(aa + 1): (aa + 1) for aa in range(self.num_classes - 1)}
        self.scales = [int(self.img_size / 544 * aa) for aa in (24, 48, 96, 192, 384)]
        self.aspect_ratios = [1, 1 / 2, 2]


        if self.mode == 'train':
            self.weight = args.resume if args.resume else '/content/drive/MyDrive/yolact/weights/backbone_res101.pth'
            # if not args.weight:
            #     # net.apply(init_weights)
            #     self.weight = init_weights(net)
        else:
            self.weight = args.weight

        self.data_root = '/content/drive/MyDrive/'

        if self.mode == 'train':
            self.train_imgs = self.data_root + 'custom_dataset/train/'
            self.train_ann = self.data_root + 'custom_dataset/train/_annotations.coco.json'
            self.train_bs = args.train_bs
            self.bs_per_gpu = args.bs_per_gpu
            self.val_interval = args.val_interval

            self.bs_factor = self.train_bs / 8
            self.lr = 0.001 * self.bs_factor
            # self.lr = 0.001
            self.warmup_init = self.lr * 0.1
            self.warmup_until = 500  # If adapted with bs_factor, inifinte loss may appear.
            self.lr_steps = tuple([int(aa / self.bs_factor) for aa in (0, 280000, 560000, 620000, 680000)])

            self.pos_iou_thre = 0.5
            self.neg_iou_thre = 0.4

            self.conf_alpha = 1
            self.bbox_alpha = 1.5
            self.mask_alpha = 6.125
            self.semantic_alpha = 1

            # The max number of masks to train for one image.
            self.masks_to_train = 200

        if self.mode in ('train', 'val'):
            self.val_imgs = self.data_root + 'custom_dataset/valid/'
            self.val_ann = self.data_root + 'custom_dataset/valid/_annotations.coco.json'
            self.val_bs = 1
            self.val_num = args.val_num
            self.coco_api = args.coco_api

        self.traditional_nms = args.traditional_nms
        self.nms_score_thre = 0.05
        self.nms_iou_thre = 0.5
        self.top_k = 200
        self.max_detections = 200

        if self.mode == 'detect':
            for k, v in vars(args).items():
                self.__setattr__(k, v)

    def print_cfg(self):
        print()
        print('-' * 30 + self.__class__.__name__ + '-' * 30)
        for k, v in vars(self).items():
            if k not in ('continuous_id', 'data_root', 'cfg'):
                print(f'{k}: {v}')
        print()


def get_config(args, mode):
    args.cuda = torch.cuda.is_available()
    args.mode = mode

    if args.cuda:
        # Set GPU id if CUDA is available
        # args.gpu_id = '0'  # Assuming using the first GPU
        if args.mode == 'train':
            # Training mode
            num_gpus = torch.cuda.device_count()
            args.bs_per_gpu = args.train_bs // num_gpus
        # else:
            # Validation or detection mode
            # assert args.gpu_id.isdigit(), 'Only one GPU can be used in val/detect mode.'
    else:
        # args.gpu_id = None
        if args.mode == 'train':
            args.bs_per_gpu = args.train_bs
            print('\n-----No GPU found, training on CPU.-----')
        else:
            print('\n-----No GPU found, validate on CPU.-----')

    # klas = 'res101_coco'
    # cfg = globals()[args.klas](args)
    cfg = res101_coco(args)

    # Print configuration
    if not args.cuda or args.mode != 'train':
        cfg.print_cfg()
    else:
        cfg.print_cfg()

    return cfg

In [2]:
import pyximport
pyximport.install()

(None, <pyximport._pyximport3.PyxImportMetaFinder at 0x7e3eceb752d0>)

In [3]:
!cythonize -a cython_nms.pyx

/usr/local/bin/cythonize: No such file or directory: 'cython_nms.pyx'


In [4]:
import cv2
import numpy as np
import random

# Warning, do not use numpy random in PyTorch multiprocessing, or the random result will be the same.

def random_mirror(img, masks, boxes):
    if random.randint(0, 1):
        _, width, _ = img.shape
        img = img[:, ::-1]
        masks = masks[:, :, ::-1]
        boxes[:, 0::2] = width - boxes[:, 2::-2]

    return img, masks, boxes


def clip_box(hw, boxes):
    boxes[:, [0, 2]] = np.clip(boxes[:, [0, 2]], a_min=0, a_max=hw[1] - 1)
    boxes[:, [1, 3]] = np.clip(boxes[:, [1, 3]], a_min=0, a_max=hw[0] - 1)
    return boxes


def remove_small_box(boxes, masks, labels, area_limit):
    box_areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    keep = box_areas > area_limit
    return boxes[keep], masks[keep], labels[keep]


def to_01_box(hw, boxes):
    boxes[:, [0, 2]] /= hw[1]
    boxes[:, [1, 3]] /= hw[0]
    return boxes


def random_brightness(img, delta=32):
    img += random.uniform(-delta, delta)

    return np.clip(img, 0., 255.)


def random_contrast(img, lower=0.7, upper=1.3):
    img *= random.uniform(lower, upper)

    return np.clip(img, 0., 255.)


def random_saturation(img, lower=0.7, upper=1.3):
    img[:, :, 1] *= random.uniform(lower, upper)
    return img


def random_hue(img, delta=15.):  # better keep 0.< delta <=30., two large delta harms the image
    img[:, :, 0] += random.uniform(-delta, delta)
    img[:, :, 0][img[:, :, 0] > 360.0] -= 360.0
    img[:, :, 0][img[:, :, 0] < 0.0] += 360.0
    return img


def photometric_distort(img):
    if random.randint(0, 1):
        img = random_brightness(img)
    if random.randint(0, 1):
        img = random_contrast(img)

    # Because of normalization, random brightness and random contrast are meanless
    # if random_saturation and random_hue do not follow.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img = random_saturation(img)
    img = random_hue(img)
    img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    img = np.clip(img, 0., 255.)

    # TODO: need a random noise aug crop
    return img


def crop_eight(ori_h, crop_h, ori_w, crop_w, img, masks, boxes, labels, keep_ratio=0.3):
    num_boxes = boxes.shape[0]
    box_x1, box_y1, box_x2, box_y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    box_areas = (box_x2 - box_x1) * (box_y2 - box_y1)

    ii = 0
    cut_out = True
    while cut_out:
        ii += 1
        if ii > 1000:
            return None, None, None, None

        random_x1 = random.randint(0, ori_w - crop_w)
        random_y1 = random.randint(0, ori_h - crop_h)

        new_x1 = np.tile(random_x1, (num_boxes, 1)).astype('float32')
        new_y1 = np.tile(random_y1, (num_boxes, 1)).astype('float32')
        new_x2 = np.tile(random_x1 + crop_w, (num_boxes, 1)).astype('float32')
        new_y2 = np.tile(random_y1 + crop_h, (num_boxes, 1)).astype('float32')

        min_x1 = np.max(np.concatenate([new_x1, box_x1.reshape(num_boxes, -1)], axis=1), axis=1)
        min_y1 = np.max(np.concatenate([new_y1, box_y1.reshape(num_boxes, -1)], axis=1), axis=1)
        max_x2 = np.min(np.concatenate([new_x2, box_x2.reshape(num_boxes, -1)], axis=1), axis=1)
        max_y2 = np.min(np.concatenate([new_y2, box_y2.reshape(num_boxes, -1)], axis=1), axis=1)

        inter_w = np.clip((max_x2 - min_x1), a_min=0, a_max=10000)
        inter_h = np.clip((max_y2 - min_y1), a_min=0, a_max=10000)

        inter_area = inter_w * inter_h
        keep = (inter_area / box_areas) > keep_ratio

        if keep.any():
            box_part = np.stack([min_x1, min_y1, max_x2, max_y2, labels], axis=0).T
            boxes_remained = box_part[keep]
            masks_remained = masks[keep]

            boxes_remained[:, [0, 2]] -= random_x1
            boxes_remained[:, [1, 3]] -= random_y1

            img_cropped = img[int(new_y1[0]): int(new_y2[0]), int(new_x1[0]): int(new_x2[0]), :]
            masks_remained = masks_remained[:, int(new_y1[0]): int(new_y2[0]), int(new_x1[0]): int(new_x2[0])]

            cut_out = False

    return img_cropped, masks_remained, boxes_remained[:, :4], boxes_remained[:, 4]


def random_crop(img, masks, boxes, labels, crop_ratio):
    if random.randint(0, 1):
        return img, masks, boxes, labels
    else:
        ori_h, ori_w, _ = img.shape
        crop_h = int(random.uniform(crop_ratio[0], crop_ratio[1]) * ori_h)
        crop_w = int(random.uniform(crop_ratio[0], crop_ratio[1]) * ori_w)

        return crop_eight(ori_h, crop_h, ori_w, crop_w, img, masks, boxes, labels)


def pad_to_square(img, masks=None, boxes=None, during_training=False):
    img_h, img_w = img.shape[:2]
    if img_h == img_w:
        return (img, masks, boxes) if during_training else img
    else:
        pad_size = max(img_h, img_w)
        pad_img = np.zeros((pad_size, pad_size, 3), dtype='float32')
        pad_img[:, :, :] = norm_mean

        if during_training:
            pad_masks = np.zeros((masks.shape[0], pad_size, pad_size), dtype='float32')

            if img_h < img_w:
                random_y1 = random.randint(0, img_w - img_h)
                pad_img[random_y1: random_y1 + img_h, :, :] = img
                pad_masks[:, random_y1: random_y1 + img_h, :] = masks
                boxes[:, [1, 3]] += random_y1

            if img_h > img_w:
                random_x1 = random.randint(0, img_h - img_w)
                pad_img[:, random_x1: random_x1 + img_w, :] = img
                pad_masks[:, :, random_x1: random_x1 + img_w] = masks
                boxes[:, [0, 2]] += random_x1

            return pad_img, pad_masks, boxes
        else:
            pad_img[0: img_h, 0: img_w, :] = img
            return pad_img


def multi_scale_resize(img, masks=None, boxes=None, resize_range=None, during_training=False):
    assert img.shape[0] == img.shape[1], 'Error, image is not square in <multi_scale_resize>'

    if during_training:
        ori_size = img.shape[0]
        resize_size = random.randint(resize_range[0], resize_range[1]) * 32
        img = cv2.resize(img, (resize_size, resize_size))
        scale = resize_size / ori_size
        boxes *= scale

        masks = masks.transpose((1, 2, 0))
        masks = cv2.resize(masks, (resize_size, resize_size))

        # OpenCV resizes a (w,h,1) array to (s,s), so fix it
        if len(masks.shape) == 2:
            masks = np.expand_dims(masks, 0)
        else:
            masks = masks.transpose((2, 0, 1))

        return img, masks, boxes
    else:
        return cv2.resize(img, (resize_range, resize_range))


def to_train_size(img, masks, boxes, labels, train_size):
    img_size = img.shape[0]

    if img_size == train_size:
        return img, masks, boxes, labels
    elif img_size < train_size:
        # pad_img = np.zeros((train_size, train_size, 3), dtype='float32')
        # pad_masks = np.zeros((masks.shape[0], train_size, train_size), dtype='float32')
        # pad_img[:, :, :] = norm_mean
        # random_y1 = random.randint(0, train_size - img_size)
        # random_x1 = random.randint(0, train_size - img_size)
        # pad_img[random_y1: random_y1 + img_size, random_x1: random_x1 + img_size, :] = img
        # pad_masks[:, random_y1: random_y1 + img_size, random_x1: random_x1 + img_size] = masks
        # boxes[:, [1, 3]] += random_y1
        # boxes[:, [0, 2]] += random_x1
        # return pad_img, pad_masks, boxes, labels
        # crop
        pass
    else:
        return crop_eight(img_size, train_size, img_size, train_size, img, masks, boxes, labels)


def normalize_and_toRGB(img):
    img = (img - norm_mean) / norm_std
    img = img[:, :, (2, 1, 0)]
    img = np.transpose(img, (2, 0, 1))
    return img


def val_aug(img, val_size):
    img = img.astype('float32')
    img = pad_to_square(img, during_training=False)
    img = multi_scale_resize(img, resize_range=val_size, during_training=False)
    img = normalize_and_toRGB(img)
    return img


def train_aug(img, masks, boxes, labels, train_size):
    # show_ann(img, masks, boxes, labels)
    img = img.astype('float32')
    img = photometric_distort(img)
    img, masks, boxes = random_mirror(img, masks, boxes)
    img, masks, boxes, labels = random_crop(img, masks, boxes, labels, crop_ratio=(0.6, 1))
    if img is None:
        return None, None, None, None
    img, masks, boxes = pad_to_square(img, masks, boxes, during_training=True)
    img, masks, boxes = multi_scale_resize(img, masks, boxes, (8, 24), during_training=True)  # multiple of 32
    img, masks, boxes, labels = to_train_size(img, masks, boxes, labels, train_size)
    if img is None:
        return None, None, None, None
    boxes = clip_box(img.shape[:2], boxes)
    boxes, masks, labels = remove_small_box(boxes, masks, labels, area_limit=20)
    if boxes.shape[0] == 0:
        return None, None, None, None
    assert boxes.shape[0] == masks.shape[0] == labels.shape[0], 'Error, unequal boxes, masks or labels number.'
    # show_ann(img, masks, boxes, labels)
    boxes = to_01_box(img.shape[:2], boxes)
    img = normalize_and_toRGB(img)

    return img, masks, boxes, labels


def show_ann(img, masks, boxes, labels):
    masks_semantic = masks * (labels[:, None, None] + 1)  # expand class_ids' shape for broadcasting
    # The color of the overlap area is different because of the '%' operation.
    masks_semantic = masks_semantic.astype('int').sum(axis=0) % 80
    color_masks = COLORS[masks_semantic].astype('uint8')
    img_u8 = img.astype('uint8')
    img_fused = cv2.addWeighted(color_masks, 0.4, img_u8, 0.6, gamma=0)

    for i in range(boxes.shape[0]):
        cv2.rectangle(img_fused, (int(boxes[i, 0]), int(boxes[i, 1])),
                      (int(boxes[i, 2]), int(boxes[i, 3])), (0, 255, 0), 1)

    print(f'\nimg shape: {img.shape}')
    print('----------------boxes----------------')
    print(boxes)
    print('----------------labels---------------')
    print(labels, '\n')
    cv2.imshow('aa', img_fused)
    cv2.waitKey()


In [5]:
# -*- coding: utf-8 -*-
import torch
from itertools import product
from math import sqrt
import numpy as np


def box_iou(box_a, box_b):
    """
    Compute the IoU of two sets of boxes.
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects,4]
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors,4]
    Return:
        jaccard overlap: (tensor) Shape: [box_a.size(0), box_b.size(0)]
    """
    use_batch = True
    if box_a.dim() == 2:
        use_batch = False
        box_a = box_a[None, ...]
        box_b = box_b[None, ...]

    (n, A), B = box_a.shape[:2], box_b.shape[1]
    # add a dimension
    box_a = box_a[:, :, None, :].expand(n, A, B, 4)
    box_b = box_b[:, None, :, :].expand(n, A, B, 4)

    max_xy = torch.min(box_a[..., 2:], box_b[..., 2:])
    min_xy = torch.max(box_a[..., :2], box_b[..., :2])
    inter = torch.clamp((max_xy - min_xy), min=0)
    inter_area = inter[..., 0] * inter[..., 1]

    area_a = (box_a[..., 2] - box_a[..., 0]) * (box_a[..., 3] - box_a[..., 1])
    area_b = (box_b[..., 2] - box_b[..., 0]) * (box_b[..., 3] - box_b[..., 1])

    out = inter_area / (area_a + area_b - inter_area)
    return out if use_batch else out.squeeze(0)


def box_iou_numpy(box_a, box_b):
    (n, A), B = box_a.shape[:2], box_b.shape[1]
    # add a dimension
    box_a = np.tile(box_a[:, :, None, :], (1, 1, B, 1))
    box_b = np.tile(box_b[:, None, :, :], (1, A, 1, 1))

    max_xy = np.minimum(box_a[..., 2:], box_b[..., 2:])
    min_xy = np.maximum(box_a[..., :2], box_b[..., :2])
    inter = np.clip((max_xy - min_xy), a_min=0, a_max=100000)
    inter_area = inter[..., 0] * inter[..., 1]

    area_a = (box_a[..., 2] - box_a[..., 0]) * (box_a[..., 3] - box_a[..., 1])
    area_b = (box_b[..., 2] - box_b[..., 0]) * (box_b[..., 3] - box_b[..., 1])

    return inter_area / (area_a + area_b - inter_area)


def match(cfg, box_gt, anchors, class_gt):
    # Convert prior boxes to the form of [xmin, ymin, xmax, ymax].
    decoded_priors = torch.cat((anchors[:, :2] - anchors[:, 2:] / 2, anchors[:, :2] + anchors[:, 2:] / 2), 1)

    overlaps = box_iou(box_gt, decoded_priors)  # (num_gts, num_achors)

    _, gt_max_i = overlaps.max(1)  # (num_gts, ) the max IoU for each gt box
    each_anchor_max, anchor_max_i = overlaps.max(0)  # (num_achors, ) the max IoU for each anchor

    each_anchor_max.index_fill_(0, gt_max_i, 2)

    # Set the index of the pair (anchor, gt) we set the overlap for above.
    for j in range(gt_max_i.size(0)):
        anchor_max_i[gt_max_i[j]] = j

    anchor_max_gt = box_gt[anchor_max_i]  # (num_achors, 4)

    conf = class_gt[anchor_max_i] + 1  # the class of the max IoU gt box for each anchor
    conf[each_anchor_max < cfg.pos_iou_thre] = -1  # label as neutral
    conf[each_anchor_max < cfg.neg_iou_thre] = 0  # label as background

    offsets = encode(anchor_max_gt, anchors)

    return offsets, conf, anchor_max_gt, anchor_max_i


def make_anchors(cfg, conv_h, conv_w, scale):
    prior_data = []
    # Iteration order is important (it has to sync up with the convout)
    for j, i in product(range(conv_h), range(conv_w)):
        # + 0.5 because priors are in center
        x = (i + 0.5) / conv_w
        y = (j + 0.5) / conv_h

        for ar in cfg.aspect_ratios:
            ar = sqrt(ar)
            w = scale * ar / cfg.img_size
            h = scale / ar / cfg.img_size

            prior_data += [x, y, w, h]

    return prior_data


def encode(matched, priors):
    variances = [0.1, 0.2]

    g_cxcy = (matched[:, :2] + matched[:, 2:]) / 2 - priors[:, :2]  # 10 * (Xg - Xa) / Wa
    g_cxcy /= (variances[0] * priors[:, 2:])  # 10 * (Yg - Ya) / Ha
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]  # 5 * log(Wg / Wa)
    g_wh = torch.log(g_wh) / variances[1]  # 5 * log(Hg / Ha)
    # return target for smooth_l1_loss
    offsets = torch.cat([g_cxcy, g_wh], 1)  # [num_priors, 4]

    return offsets


def sanitize_coordinates(_x1, _x2, img_size, padding=0):

    _x1 = _x1 * img_size
    _x2 = _x2 * img_size

    x1 = torch.min(_x1, _x2)
    x2 = torch.max(_x1, _x2)
    x1 = torch.clamp(x1 - padding, min=0)
    x2 = torch.clamp(x2 + padding, max=img_size)

    return x1, x2


def sanitize_coordinates_numpy(_x1, _x2, img_size, padding=0):
    _x1 = _x1 * img_size
    _x2 = _x2 * img_size

    x1 = np.minimum(_x1, _x2)
    x2 = np.maximum(_x1, _x2)
    x1 = np.clip(x1 - padding, a_min=0, a_max=1000000)
    x2 = np.clip(x2 + padding, a_min=0, a_max=img_size)

    return x1, x2


def crop_box(masks, boxes, padding=1):

    h, w, n = masks.size()
    x1, x2 = sanitize_coordinates(boxes[:, 0], boxes[:, 2], w, padding)
    y1, y2 = sanitize_coordinates(boxes[:, 1], boxes[:, 3], h, padding)

    rows = torch.arange(w, device=masks.device, dtype=x1.dtype).view(1, -1, 1).expand(h, w, n)
    cols = torch.arange(h, device=masks.device, dtype=x1.dtype).view(-1, 1, 1).expand(h, w, n)

    masks_left = rows >= x1.view(1, 1, -1)
    masks_right = rows < x2.view(1, 1, -1)
    masks_up = cols >= y1.view(1, 1, -1)
    masks_down = cols < y2.view(1, 1, -1)

    crop_mask = masks_left * masks_right * masks_up * masks_down

    return masks * crop_mask.float()


def crop_numpy(masks, boxes, padding=1):
    h, w, n = masks.shape
    x1, x2 = sanitize_coordinates_numpy(boxes[:, 0], boxes[:, 2], w, padding)
    y1, y2 = sanitize_coordinates_numpy(boxes[:, 1], boxes[:, 3], h, padding)

    rows = np.tile(np.arange(w)[None, :, None], (h, 1, n))
    cols = np.tile(np.arange(h)[:, None, None], (1, w, n))

    masks_left = rows >= (x1.reshape(1, 1, -1))
    masks_right = rows < (x2.reshape(1, 1, -1))
    masks_up = cols >= (y1.reshape(1, 1, -1))
    masks_down = cols < (y2.reshape(1, 1, -1))

    crop_mask = masks_left * masks_right * masks_up * masks_down

    return masks * crop_mask


def mask_iou(mask1, mask2):

    intersection = torch.matmul(mask1, mask2.t())
    area1 = torch.sum(mask1, dim=1).reshape(1, -1)
    area2 = torch.sum(mask2, dim=1).reshape(1, -1)
    union = (area1.t() + area2) - intersection
    ret = intersection / union

    return ret.cpu()


In [6]:
import os.path as osp
import torch
import torch.utils.data as data
import cv2
import glob
import numpy as np
from pycocotools.coco import COCO


def train_collate(batch):
    imgs, targets, masks = [], [], []
    valid_batch = [aa for aa in batch if aa[0] is not None]

    lack_len = len(batch) - len(valid_batch)
    if lack_len > 0:
        for i in range(lack_len):
            valid_batch.append(valid_batch[i])

    for sample in valid_batch:
        imgs.append(torch.tensor(sample[0], dtype=torch.float32))
        targets.append(torch.tensor(sample[1], dtype=torch.float32))
        masks.append(torch.tensor(sample[2], dtype=torch.float32))

    return torch.stack(imgs, 0), targets, masks


def val_collate(batch):
    imgs = torch.tensor(batch[0][0], dtype=torch.float32).unsqueeze(0)
    targets = torch.tensor(batch[0][1], dtype=torch.float32)
    masks = torch.tensor(batch[0][2], dtype=torch.float32)
    return imgs, targets, masks, batch[0][3], batch[0][4]


def detect_collate(batch):
    imgs = torch.tensor(batch[0][0], dtype=torch.float32).unsqueeze(0)
    return imgs, batch[0][1], batch[0][2]


def detect_onnx_collate(batch):
    return batch[0][0][None, :], batch[0][1], batch[0][2]


class COCODetection(data.Dataset):
    def __init__(self, cfg, mode='train'):
        self.mode = mode
        self.cfg = cfg

        if mode in ('train', 'val'):
            self.image_path = cfg.train_imgs if mode == 'train' else cfg.val_imgs
            self.coco = COCO(cfg.train_ann if mode == 'train' else cfg.val_ann)
            self.ids = list(self.coco.imgToAnns.keys())
        elif mode == 'detect':
            self.image_path = glob.glob(cfg.image + '/*.jpg')
            self.image_path.sort()

        self.continuous_id = cfg.continuous_id

    def __getitem__(self, index):
        if self.mode == 'detect':
            img_name = self.image_path[index]
            img_origin = cv2.imread(img_name)
            img_normed = val_aug(img_origin, self.cfg.img_size)
            return img_normed, img_origin, img_name.split(osp.sep)[-1]
        else:
            img_id = self.ids[index]
            ann_ids = self.coco.getAnnIds(imgIds=img_id)

            # 'target' includes {'segmentation', 'area', iscrowd', 'image_id', 'bbox', 'category_id'}
            target = self.coco.loadAnns(ann_ids)
            target = [aa for aa in target if not aa['iscrowd']]

            file_name = self.coco.loadImgs(img_id)[0]['file_name']

            img_path = osp.join(self.image_path, file_name)
            assert osp.exists(img_path), f'Image path does not exist: {img_path}'

            img = cv2.imread(img_path)
            height, width, _ = img.shape

            assert len(target) > 0, 'No annotation in this image!'
            box_list, mask_list, label_list = [], [], []

            for aa in target:
                bbox = aa['bbox']

                # When training, some boxes are wrong, ignore them.
                if self.mode == 'train':
                    if bbox[0] < 0 or bbox[1] < 0 or bbox[2] < 4 or bbox[3] < 4:
                        continue

                x1y1x2y2_box = np.array([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
                category = self.continuous_id[aa['category_id']] - 1

                box_list.append(x1y1x2y2_box)
                mask_list.append(self.coco.annToMask(aa))
                label_list.append(category)

            if len(box_list) > 0:
                boxes = np.array(box_list)
                masks = np.stack(mask_list, axis=0)
                labels = np.array(label_list)
                assert masks.shape == (boxes.shape[0], height, width), 'Unmatched annotations.'

                if self.mode == 'train':
                    img, masks, boxes, labels = train_aug(img, masks, boxes, labels, self.cfg.img_size)
                    if img is None:
                        return None, None, None
                    else:
                        boxes = np.hstack((boxes, np.expand_dims(labels, axis=1)))
                        return img, boxes, masks
                elif self.mode == 'val':
                    img = val_aug(img, self.cfg.img_size)
                    boxes = boxes / np.array([width, height, width, height])  # to 0~1 scale
                    boxes = np.hstack((boxes, np.expand_dims(labels, axis=1)))
                    return img, boxes, masks, height, width
            else:
                if self.mode == 'val':
                    raise RuntimeError('Error, no valid object in this image.')
                else:
                    print(f'No valid object in image: {img_id}. Use a repeated image in this batch.')
                    return None, None, None

    def __len__(self):
        if self.mode == 'train':
            return len(self.ids)
        elif self.mode == 'val':
            return len(self.ids) if self.cfg.val_num == -1 else min(self.cfg.val_num, len(self.ids))
        elif self.mode == 'detect':
            return len(self.image_path)


In [7]:
!pip install terminaltables
import glob
import torch
import os
import numpy as np
import pycocotools
import json
from terminaltables import AsciiTable
from collections import OrderedDict


class ProgressBar:
    def __init__(self, length, max_val):
        self.max_val = max_val
        self.length = length
        self.cur_val = 0

        self.cur_num_bars = -1
        self.update_str()

    def update_str(self):
        num_bars = int(self.length * (self.cur_val / self.max_val))

        if num_bars != self.cur_num_bars:
            self.cur_num_bars = num_bars
            self.string = '█' * num_bars + '░' * (self.length - num_bars)

    def get_bar(self, new_val):
        self.cur_val = new_val

        if self.cur_val > self.max_val:
            self.cur_val = self.max_val
        self.update_str()
        return self.string


def save_best(net, mask_map, cfg_name, step):
    weight = glob.glob('/content/drive/MyDrive/yolact/weights/best*')
    weight = [aa for aa in weight if cfg_name in aa]
    assert len(weight) <= 1, 'Error, multiple best weight found.'
    best_mask_map = float(weight[0].split('/')[-1].split('_')[1]) if weight else 0.

    if mask_map >= best_mask_map:
        if weight:
            os.remove(weight[0])  # remove the last best model

        print(f'\nSaving the best model as \'best_{mask_map}_{cfg_name}_{step}.pth\'.\n')
        torch.save(net.state_dict(), f'/content/drive/MyDrive/yolact/weights/best_{mask_map}_{cfg_name}_{step}.pth')


def save_latest(net, cfg_name, step):
    weight = glob.glob('/content/drive/MyDrive/yolact/weights/latest*')
    weight = [aa for aa in weight if cfg_name in aa]
    assert len(weight) <= 1, 'Error, multiple latest weight found.'
    if weight:
        os.remove(weight[0])

    print(f'\nSaving the latest model as \'latest_{cfg_name}_{step}.pth\'.\n')
    torch.save(net.state_dict(), f'/content/drive/MyDrive/yolact/weights/latest_{cfg_name}_{step}.pth')


class MakeJson:
    def __init__(self):
        self.bbox_data = []
        self.mask_data = []
        self.coco_cats = {}

        for coco_id, real_id in COCO_LABEL_MAP.items():
            class_id = real_id - 1
            self.coco_cats[class_id] = coco_id

    def add_bbox(self, image_id: int, category_id: int, bbox: list, score: float):
        """ Note that bbox should be a list or tuple of (x1, y1, x2, y2) """
        bbox = [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]]

        # Round to the nearest 10th to avoid huge file sizes, as COCO suggests
        bbox = [round(float(x) * 10) / 10 for x in bbox]

        self.bbox_data.append({'image_id': int(image_id),
                               'category_id': self.coco_cats[int(category_id)],
                               'bbox': bbox,
                               'score': float(score)})

    def add_mask(self, image_id: int, category_id: int, segmentation: np.ndarray, score: float):
        """ The segmentation should be the full mask, the size of the image and with size [h, w]. """
        rle = pycocotools.mask.encode(np.asfortranarray(segmentation.astype(np.uint8)))
        rle['counts'] = rle['counts'].decode('ascii')  # json.dump doesn't like bytes strings

        self.mask_data.append({'image_id': int(image_id),
                               'category_id': self.coco_cats[int(category_id)],
                               'segmentation': rle,
                               'score': float(score)})

    def dump(self):
        dump_arguments = [(self.bbox_data, f'/content/drive/MyDrive/yolact/results/bbox_detections.json'),
                          (self.mask_data, f'/content/drive/MyDrive/yolact/results/mask_detections.json')]

        for data, path in dump_arguments:
            with open(path, 'w') as f:
                json.dump(data, f)


class APDataObject:
    """Stores all the information necessary to calculate the AP for one IoU and one class."""

    def __init__(self):
        self.data_points = []
        self.num_gt_positives = 0

    def push(self, score: float, is_true: bool):
        self.data_points.append((score, is_true))

    def add_gt_positives(self, num_positives: int):
        self.num_gt_positives += num_positives

    def is_empty(self) -> bool:
        return len(self.data_points) == 0 and self.num_gt_positives == 0

    def get_ap(self) -> float:
        if self.num_gt_positives == 0:
            return 0

        # Sort descending by score
        self.data_points.sort(key=lambda x: -x[0])

        precisions = []
        recalls = []
        num_true = 0
        num_false = 0

        # Compute the precision-recall curve. The x axis is recalls and the y axis precisions.
        for datum in self.data_points:
            # datum[1] is whether the detection a true or false positive
            if datum[1]:
                num_true += 1
            else:
                num_false += 1

            precision = num_true / (num_true + num_false)
            recall = num_true / self.num_gt_positives

            precisions.append(precision)
            recalls.append(recall)

        # Smooth the curve by computing [max(precisions[i:]) for i in range(len(precisions))]
        # Basically, remove any temporary dips from the curve.
        # At least that's what I think, idk. COCOEval did it so I do too.
        for i in range(len(precisions) - 1, 0, -1):
            if precisions[i] > precisions[i - 1]:
                precisions[i - 1] = precisions[i]

        # Compute the integral of precision(recall) d_recall from recall=0->1 using fixed-length riemann summation with 101 bars.
        y_range = [0] * 101  # idx 0 is recall == 0.0 and idx 100 is recall == 1.00
        x_range = np.array([x / 100 for x in range(101)])
        recalls = np.array(recalls)

        # I realize this is weird, but all it does is find the nearest precision(x) for a given x in x_range.
        # Basically, if the closest recall we have to 0.01 is 0.009 this sets precision(0.01) = precision(0.009).
        # I approximate the integral this way, because that's how COCOEval does it.
        indices = np.searchsorted(recalls, x_range, side='left')
        for bar_idx, precision_idx in enumerate(indices):
            if precision_idx < len(precisions):
                y_range[bar_idx] = precisions[precision_idx]

        # Finally compute the riemann sum to get our integral.
        # avg([precision(x) for x in 0:0.01:1])
        return sum(y_range) / len(y_range)


def prep_metrics(ap_data, ids_p, classes_p, boxes_p, masks_p, gt, gt_masks, height, width, iou_thres):
    gt_boxes = gt[:, :4]
    gt_boxes[:, [0, 2]] *= width
    gt_boxes[:, [1, 3]] *= height
    gt_classes = gt[:, 4].int().tolist()
    gt_masks = gt_masks.reshape(-1, height * width)
    masks_p = masks_p.reshape(-1, height * width)

    mask_iou_cache = mask_iou(masks_p, gt_masks)
    bbox_iou_cache = box_iou(boxes_p.float(), gt_boxes.float()).cpu()

    for _class in set(ids_p + gt_classes):
        num_gt_per_class = gt_classes.count(_class)

        for iouIdx in range(len(iou_thres)):
            iou_threshold = iou_thres[iouIdx]

            for iou_type, iou_func in zip(['box', 'mask'], [bbox_iou_cache, mask_iou_cache]):
                gt_used = [False] * len(gt_classes)
                ap_obj = ap_data[iou_type][iouIdx][_class]
                ap_obj.add_gt_positives(num_gt_per_class)

                for i, pred_class in enumerate(ids_p):
                    if pred_class != _class:
                        continue

                    max_iou_found = iou_threshold
                    max_match_idx = -1
                    for j, gt_class in enumerate(gt_classes):
                        if gt_used[j] or gt_class != _class:
                            continue

                        iou = iou_func[i, j].item()

                        if iou > max_iou_found:
                            max_iou_found = iou
                            max_match_idx = j

                    if max_match_idx >= 0:
                        gt_used[max_match_idx] = True
                        ap_obj.push(classes_p[i], True)
                    else:
                        ap_obj.push(classes_p[i], False)


def calc_map(ap_data, iou_thres, num_classes, step):
    print('\nCalculating mAP...')
    aps = [{'box': [], 'mask': []} for _ in iou_thres]

    for _class in range(num_classes):
        for iou_idx in range(len(iou_thres)):
            for iou_type in ('box', 'mask'):
                ap_obj = ap_data[iou_type][iou_idx][_class]

                if not ap_obj.is_empty():
                    aps[iou_idx][iou_type].append(ap_obj.get_ap())

    all_maps = {'box': OrderedDict(), 'mask': OrderedDict()}

    for iou_type in ('box', 'mask'):
        all_maps[iou_type]['all'] = 0  # Make this first in the ordereddict

        for i, threshold in enumerate(iou_thres):
            mAP = sum(aps[i][iou_type]) / len(aps[i][iou_type]) * 100 if len(aps[i][iou_type]) > 0 else 0
            all_maps[iou_type][int(threshold * 100)] = mAP

        all_maps[iou_type]['all'] = (sum(all_maps[iou_type].values()) / (len(all_maps[iou_type].values()) - 1))

    row1 = list(all_maps['box'].keys())
    row1.insert(0, f'{step // 1000}k' if step else '')

    row2 = list(all_maps['box'].values())
    row2 = [round(aa, 2) for aa in row2]
    row2.insert(0, 'box')

    row3 = list(all_maps['mask'].values())
    row3 = [round(aa, 2) for aa in row3]
    row3.insert(0, 'mask')

    table = [row1, row2, row3]
    table = AsciiTable(table)
    return table.table, row2, row3


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.fft as fft
import pdb

import torch.nn.init as init


class PredictionModule(nn.Module):
    def __init__(self, cfg, coef_dim=32):
        super().__init__()
        self.num_classes = cfg.num_classes
        self.coef_dim = coef_dim

        self.upfeature = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, padding=1),
                                       nn.ReLU(inplace=True))
        self.bbox_layer = nn.Conv2d(256, len(cfg.aspect_ratios) * 4, kernel_size=3, padding=1)
        self.conf_layer = nn.Conv2d(256, len(cfg.aspect_ratios) * self.num_classes, kernel_size=3, padding=1)
        self.coef_layer = nn.Sequential(nn.Conv2d(256, len(cfg.aspect_ratios) * self.coef_dim,
                                                  kernel_size=3, padding=1),
                                        nn.Tanh())

    def forward(self, x):
        x = self.upfeature(x)
        conf = self.conf_layer(x).permute(0, 2, 3, 1).reshape(x.size(0), -1, self.num_classes)
        box = self.bbox_layer(x).permute(0, 2, 3, 1).reshape(x.size(0), -1, 4)
        coef = self.coef_layer(x).permute(0, 2, 3, 1).reshape(x.size(0), -1, self.coef_dim)
        return conf, box, coef


class ProtoNet(nn.Module):
    def __init__(self, coef_dim):
        super().__init__()
        self.proto1 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(inplace=True))
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.proto2 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(256, coef_dim, kernel_size=1, stride=1),
                                    nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.proto1(x)
        x = self.upsample(x)
        x = self.proto2(x)
        return x


class DFFN(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.p3_p4_p5_conv = nn.ModuleList([nn.Conv2d(x, 256, kernel_size=1) for x in self.in_channels])

        self.n3_conv = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.n4_conv = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.n5_conv = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.m3_down = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)
        self.m4_down = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)
        # Note the below line compare WHEN TRAINING!!! according to the minimal implementation ReLU is applied
        # self.n6_down = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1), nn.ReLU(inplace=True))
        self.n6_down = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)
        self.n7_down = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)

    def forward(self, backbone_outs):
        # Calculate P5 from c5 in this implementation backbone_outs[2]==c5 crop
        p5 = self.p3_p4_p5_conv[2](backbone_outs[2])

        # Calculate P4 c4==backbone_outs[1]
        p4_upsample = F.interpolate(p5, size=(34,34), mode='bilinear', align_corners=False)
        p4_conv = self.p3_p4_p5_conv[1](backbone_outs[1])
        p4 = p4_upsample + p4_conv

        # Calculate P3 c3==backbone_outs[0], c2 is already removed
        p3_upsample = F.interpolate(p4, size=(68,68), mode='bilinear', align_corners=False)
        p3_conv = self.p3_p4_p5_conv[0](backbone_outs[0])
        p3 = p3_upsample + p3_conv

        # Calculate N3
        n3 = self.n3_conv(p3)

        # Calculate M3 and N4
        m3 = self.m3_down(n3)
        n4 = self.n4_conv(p4 + m3)

        # Calculate M4 and N5
        m4 = self.m4_down(n4)
        n5 = self.n5_conv(p5 + m4)

        # Calculate N6 and N7
        n6 = self.n6_down(n5)
        n7 = self.n7_down(n6)

        dffn_outs = []
        dffn_outs = n3, n4, n5, n6, n7

        return dffn_outs


class DualDomainAttentionModule(nn.Module):
    def __init__(self, C = 256, reduction=16):
        # super(DualDomainAttentionModule, self).__init__()
        super().__init__()
        self.reduction = reduction
        self.fc1 = nn.Linear(C, C // reduction)
        self.fc2 = nn.Linear(C // reduction, C)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        dct = fft.rfft2(x, dim=(-2, -1))
        dct = dct[..., :2, :2]  # Keep only the low-frequency components
        dct = torch.norm(dct, dim=(-2, -1))  # Compute the magnitude of the complex numbers
        dct = dct.view(dct.size(0), -1)  # Flatten the features

        y0 = F.relu(self.fc1(dct))
        y1 = self.fc2(y0)
        weights = self.sigmoid(y1)

        out = x * weights.unsqueeze(-1).unsqueeze(-1)

        return out


class FPN(nn.Module):
    def __init__(self, in_channels):
        super().__init__()

        self.dffn = DFFN(in_channels)
        self.ddam = DualDomainAttentionModule()

    def forward(self, backbone_outs):
        dffn_outs = self.dffn(backbone_outs)

        ddam_outs = []
        for dffn_out in dffn_outs:
            ddam_out = self.ddam(dffn_out)
            ddam_outs.append(ddam_out)

        p3, p4, p5, p6, p7 = ddam_outs
        return p3, p4, p5, p6, p7



class Yolact(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.coef_dim = 32

        self.backbone = ResNet(layers=(3, 4, 23, 3))
        self.fpn = FPN(in_channels=(512, 1024, 2048))

        self.proto_net = ProtoNet(coef_dim=self.coef_dim)
        self.prediction_layers = PredictionModule(cfg, coef_dim=self.coef_dim)

        self.anchors = []
        fpn_fm_shape = [math.ceil(cfg.img_size / stride) for stride in (8, 16, 32, 64, 128)]
        for i, size in enumerate(fpn_fm_shape):
            self.anchors += make_anchors(self.cfg, size, size, self.cfg.scales[i])

        if cfg.mode == 'train':
            self.semantic_seg_conv = nn.Conv2d(256, cfg.num_classes - 1, kernel_size=1)

        # init weights, backbone weights will be covered later
        for name, module in self.named_modules():
            if isinstance(module, nn.Conv2d):
                nn.init.xavier_uniform_(module.weight.data)

                if module.bias is not None:
                    module.bias.data.zero_()

    def load_weights(self, weight, cuda):
        if cuda:
            state_dict = torch.load(weight)
        else:
            state_dict = torch.load(weight, map_location='cpu')

        for key in list(state_dict.keys()):
            if self.cfg.mode != 'train' and key.startswith('semantic_seg_conv'):
                del state_dict[key]

        self.load_state_dict(state_dict, strict=True)
        print(f'Model loaded with {weight}.\n')
        print(f'Number of all parameters: {sum([p.numel() for p in self.parameters()])}\n')

    def init_weights(self, model):
        for name, param in model.named_parameters():
            if 'bias' in name:
                init.constant_(param, 0.0)
            elif 'weight' in name:
                if '.conv.' in name:
                    init.xavier_normal_(param)
                elif '.bn.' in name:
                    init.constant_(param, 1.0)
                elif '.bias.' in name:
                    init.constant_(param, 0.0)

        print(f'Model initialized.\n')
        # print(f'Number of all parameters: {sum([p.numel() for p in self.parameters()])}\n')


    # Forward pass
    def forward(self, img, box_classes=None, masks_gt=None):

        outs = self.backbone(img)

        outs = self.fpn(outs[1:4])

        proto_out = self.proto_net(outs[0])  # feature map P3
        proto_out = proto_out.permute(0, 2, 3, 1).contiguous()

        class_pred, box_pred, coef_pred = [], [], []

        for aa in outs:
            class_p, box_p, coef_p = self.prediction_layers(aa)
            class_pred.append(class_p)
            box_pred.append(box_p)
            coef_pred.append(coef_p)

        class_pred = torch.cat(class_pred, dim=1)
        box_pred = torch.cat(box_pred, dim=1)
        coef_pred = torch.cat(coef_pred, dim=1)

        if self.training:
            seg_pred = self.semantic_seg_conv(outs[0])
            return self.compute_loss(class_pred, box_pred, coef_pred, proto_out, seg_pred, box_classes, masks_gt)
        else:
            class_pred = F.softmax(class_pred, -1)
            return class_pred, box_pred, coef_pred, proto_out

    def compute_loss(self, class_p, box_p, coef_p, proto_p, seg_p, box_class, mask_gt):
        device = class_p.device
        class_gt = [None] * len(box_class)
        batch_size = box_p.size(0)

        if isinstance(self.anchors, list):
            self.anchors = torch.tensor(self.anchors, device=device).reshape(-1, 4)

        num_anchors = self.anchors.shape[0]

        all_offsets = torch.zeros((batch_size, num_anchors, 4), dtype=torch.float32, device=device)
        conf_gt = torch.zeros((batch_size, num_anchors), dtype=torch.int64, device=device)
        anchor_max_gt = torch.zeros((batch_size, num_anchors, 4), dtype=torch.float32, device=device)
        anchor_max_i = torch.zeros((batch_size, num_anchors), dtype=torch.int64, device=device)

        for i in range(batch_size):
            box_gt = box_class[i][:, :-1]
            class_gt[i] = box_class[i][:, -1].long()

            all_offsets[i], conf_gt[i], anchor_max_gt[i], anchor_max_i[i] = match(self.cfg, box_gt,
                                                                                  self.anchors, class_gt[i])

        assert (not all_offsets.requires_grad) and (not conf_gt.requires_grad) and \
               (not anchor_max_i.requires_grad), 'Incorrect computation graph, check the grad.'

        pos_bool = conf_gt > 0

        loss_c = self.category_loss(class_p, conf_gt, pos_bool)
        loss_b = self.box_loss(box_p, all_offsets, pos_bool)
        loss_m = self.lincomb_mask_loss(pos_bool, anchor_max_i, coef_p, proto_p, mask_gt, anchor_max_gt)
        loss_s = self.semantic_seg_loss(seg_p, mask_gt, class_gt)
        return loss_c, loss_b, loss_m, loss_s

    def category_loss(self, class_p, conf_gt, pos_bool, np_ratio=3):
        batch_conf = class_p.reshape(-1, self.cfg.num_classes)

        batch_conf_max = batch_conf.max()
        mark = torch.log(torch.sum(torch.exp(batch_conf - batch_conf_max), 1)) + batch_conf_max - batch_conf[:, 0]

        mark = mark.reshape(class_p.size(0), -1)
        mark[pos_bool] = 0  # filter out pos boxes
        mark[conf_gt < 0] = 0  # filter out neutrals (conf_gt = -1)

        _, idx = mark.sort(1, descending=True)
        _, idx_rank = idx.sort(1)

        num_pos = pos_bool.long().sum(1, keepdim=True)
        num_neg = torch.clamp(np_ratio * num_pos, max=pos_bool.size(1) - 1)
        neg_bool = idx_rank < num_neg.expand_as(idx_rank)

        neg_bool[pos_bool] = 0
        neg_bool[conf_gt < 0] = 0  # Filter out neutrals

        class_p_mined = class_p[(pos_bool + neg_bool)].reshape(-1, self.cfg.num_classes)
        class_gt_mined = conf_gt[(pos_bool + neg_bool)]

        return self.cfg.conf_alpha * F.cross_entropy(class_p_mined, class_gt_mined, reduction='sum') / num_pos.sum()

    def box_loss(self, box_p, all_offsets, pos_bool):
        num_pos = pos_bool.sum()
        pos_box_p = box_p[pos_bool, :]
        pos_offsets = all_offsets[pos_bool, :]

        return self.cfg.bbox_alpha * F.smooth_l1_loss(pos_box_p, pos_offsets, reduction='sum') / num_pos

    def lincomb_mask_loss(self, pos_bool, anchor_max_i, coef_p, proto_p, mask_gt, anchor_max_gt):
        proto_h, proto_w = proto_p.shape[1:3]
        total_pos_num = pos_bool.sum()
        loss_m = 0
        for i in range(coef_p.size(0)):
            # downsample the gt mask to the size of 'proto_p'
            downsampled_masks = F.interpolate(mask_gt[i].unsqueeze(0), (proto_h, proto_w), mode='bilinear',
                                              align_corners=False).squeeze(0)
            downsampled_masks = downsampled_masks.permute(1, 2, 0).contiguous()
            # binarize the gt mask because of the downsample operation
            downsampled_masks = downsampled_masks.gt(0.5).float()

            pos_anchor_i = anchor_max_i[i][pos_bool[i]]
            pos_anchor_box = anchor_max_gt[i][pos_bool[i]]
            pos_coef = coef_p[i][pos_bool[i]]

            if pos_anchor_i.size(0) == 0:
                continue

            # If exceeds the number of masks for training, select a random subset
            old_num_pos = pos_coef.size(0)
            if old_num_pos > self.cfg.masks_to_train:
                perm = torch.randperm(pos_coef.size(0))
                select = perm[:self.cfg.masks_to_train]
                pos_coef = pos_coef[select]
                pos_anchor_i = pos_anchor_i[select]
                pos_anchor_box = pos_anchor_box[select]

            num_pos = pos_coef.size(0)

            pos_mask_gt = downsampled_masks[:, :, pos_anchor_i]

            # @ means dot product
            mask_p = torch.sigmoid(proto_p[i] @ pos_coef.t())
            mask_p = crop(mask_p, pos_anchor_box)  # pos_anchor_box.shape: (num_pos, 4)
            # TODO: grad out of gt box is 0, should it be modified?
            # TODO: need an upsample before computing loss?
            mask_loss = F.binary_cross_entropy(torch.clamp(mask_p, 0, 1), pos_mask_gt, reduction='none')

            anchor_area = (pos_anchor_box[:, 2] - pos_anchor_box[:, 0]) * (pos_anchor_box[:, 3] - pos_anchor_box[:, 1])
            mask_loss = mask_loss.sum(dim=(0, 1)) / anchor_area

            if old_num_pos > num_pos:
                mask_loss *= old_num_pos / num_pos

            loss_m += torch.sum(mask_loss)

        return self.cfg.mask_alpha * loss_m / proto_h / proto_w / total_pos_num

    def semantic_seg_loss(self, segmentation_p, mask_gt, class_gt):
        # Note classes here exclude the background class, so num_classes = cfg.num_classes - 1
        batch_size, num_classes, mask_h, mask_w = segmentation_p.size()
        loss_s = 0

        for i in range(batch_size):
            cur_segment = segmentation_p[i]
            cur_class_gt = class_gt[i]

            downsampled_masks = F.interpolate(mask_gt[i].unsqueeze(0), (mask_h, mask_w), mode='bilinear',
                                              align_corners=False).squeeze(0)
            downsampled_masks = downsampled_masks.gt(0.5).float()

            # Construct Semantic Segmentation
            segment_gt = torch.zeros_like(cur_segment, requires_grad=False)
            for j in range(downsampled_masks.size(0)):
                segment_gt[cur_class_gt[j]] = torch.max(segment_gt[cur_class_gt[j]], downsampled_masks[j])

            loss_s += F.binary_cross_entropy_with_logits(cur_segment, segment_gt, reduction='sum')

        return self.cfg.semantic_alpha * loss_s / mask_h / mask_w / batch_size

In [10]:
import torch.nn.functional as F
import cv2
import torch
import numpy as np
# from cython_nms import nms as cnms
import pdb


def fast_nms(box_thre, coef_thre, class_thre, cfg):
    class_thre, idx = class_thre.sort(1, descending=True)  # [80, 64 (the number of kept boxes)]

    idx = idx[:, :cfg.top_k]
    class_thre = class_thre[:, :cfg.top_k]

    num_classes, num_dets = idx.size()
    box_thre = box_thre[idx.reshape(-1), :].reshape(num_classes, num_dets, 4)  # [80, 64, 4]
    coef_thre = coef_thre[idx.reshape(-1), :].reshape(num_classes, num_dets, -1)  # [80, 64, 32]

    iou = box_iou(box_thre, box_thre)
    iou.triu_(diagonal=1)
    iou_max, _ = iou.max(dim=1)

    # Now just filter out the ones higher than the threshold
    keep = (iou_max <= cfg.nms_iou_thre)

    # Assign each kept detection to its corresponding class
    class_ids = torch.arange(num_classes, device=box_thre.device)[:, None].expand_as(keep)

    class_ids, box_nms, coef_nms, class_nms = class_ids[keep], box_thre[keep], coef_thre[keep], class_thre[keep]

    # Only keep the top cfg.max_num_detections highest scores across all classes
    class_nms, idx = class_nms.sort(0, descending=True)

    idx = idx[:cfg.max_detections]
    class_nms = class_nms[:cfg.max_detections]

    class_ids = class_ids[idx]
    box_nms = box_nms[idx]
    coef_nms = coef_nms[idx]

    return box_nms, coef_nms, class_ids, class_nms


def fast_nms_numpy(box_thre, coef_thre, class_thre, cfg):
    # descending sort
    idx = np.argsort(-class_thre, axis=1)
    class_thre = np.sort(class_thre, axis=1)[:, ::-1]

    idx = idx[:, :cfg.top_k]
    class_thre = class_thre[:, :cfg.top_k]

    num_classes, num_dets = idx.shape
    box_thre = box_thre[idx.reshape(-1), :].reshape(num_classes, num_dets, 4)  # [80, 64, 4]
    coef_thre = coef_thre[idx.reshape(-1), :].reshape(num_classes, num_dets, -1)  # [80, 64, 32]

    iou = box_iou_numpy(box_thre, box_thre)
    iou = np.triu(iou, k=1)
    iou_max = np.max(iou, axis=1)

    # Now just filter out the ones higher than the threshold
    keep = (iou_max <= cfg.nms_iou_thre)

    # Assign each kept detection to its corresponding class
    class_ids = np.tile(np.arange(num_classes)[:, None], (1, keep.shape[1]))

    class_ids, box_nms, coef_nms, class_nms = class_ids[keep], box_thre[keep], coef_thre[keep], class_thre[keep]

    # Only keep the top cfg.max_num_detections highest scores across all classes
    idx = np.argsort(-class_nms, axis=0)
    class_nms = np.sort(class_nms, axis=0)[::-1]

    idx = idx[:cfg.max_detections]
    class_nms = class_nms[:cfg.max_detections]

    class_ids = class_ids[idx]
    box_nms = box_nms[idx]
    coef_nms = coef_nms[idx]

    return box_nms, coef_nms, class_ids, class_nms


def traditional_nms(boxes, masks, scores, cfg):
    num_classes = scores.size(0)

    idx_lst, cls_lst, scr_lst = [], [], []

    # Multiplying by max_size is necessary because of how cnms computes its area and intersections
    boxes = boxes * cfg.img_size

    for _cls in range(num_classes):
        cls_scores = scores[_cls, :]
        conf_mask = cls_scores > cfg.nms_score_thre
        idx = torch.arange(cls_scores.size(0), device=boxes.device)

        cls_scores = cls_scores[conf_mask]
        idx = idx[conf_mask]

        if cls_scores.size(0) == 0:
            continue

        preds = torch.cat([boxes[conf_mask], cls_scores[:, None]], dim=1).cpu().numpy()
        keep = cnms(preds, cfg.nms_iou_thre)
        keep = torch.tensor(keep, device=boxes.device).long()

        idx_lst.append(idx[keep])
        cls_lst.append(keep * 0 + _cls)
        scr_lst.append(cls_scores[keep])

    idx = torch.cat(idx_lst, dim=0)
    class_ids = torch.cat(cls_lst, dim=0)
    scores = torch.cat(scr_lst, dim=0)

    scores, idx2 = scores.sort(0, descending=True)
    idx2 = idx2[:cfg.max_detections]
    scores = scores[:cfg.max_detections]

    idx = idx[idx2]
    class_ids = class_ids[idx2]

    # Undo the multiplication above
    return boxes[idx] / cfg.img_size, masks[idx], class_ids, scores


def nms(class_pred, box_pred, coef_pred, proto_out, anchors, cfg):
    class_p = class_pred.squeeze()  # [19248, 81]
    box_p = box_pred.squeeze()  # [19248, 4]
    coef_p = coef_pred.squeeze()  # [19248, 32]
    proto_p = proto_out.squeeze()  # [138, 138, 32]

    if isinstance(anchors, list):
        anchors = torch.tensor(anchors, device=class_p.device).reshape(-1, 4)

    class_p = class_p.transpose(1, 0).contiguous()  # [81, 19248]

    # exclude the background class
    class_p = class_p[1:, :]
    # get the max score class of 19248 predicted boxes
    class_p_max, _ = torch.max(class_p, dim=0)  # [19248]

    # filter predicted boxes according the class score
    keep = (class_p_max > cfg.nms_score_thre)
    class_thre = class_p[:, keep]
    box_thre, anchor_thre, coef_thre = box_p[keep, :], anchors[keep, :], coef_p[keep, :]

    # decode boxes
    box_thre = torch.cat((anchor_thre[:, :2] + box_thre[:, :2] * 0.1 * anchor_thre[:, 2:],
                          anchor_thre[:, 2:] * torch.exp(box_thre[:, 2:] * 0.2)), 1)
    box_thre[:, :2] -= box_thre[:, 2:] / 2
    box_thre[:, 2:] += box_thre[:, :2]

    box_thre = torch.clip(box_thre, min=0., max=1.)

    if class_thre.shape[1] == 0:
        return None, None, None, None, None
    else:
        if not cfg.traditional_nms:
            box_thre, coef_thre, class_ids, class_thre = fast_nms(box_thre, coef_thre, class_thre, cfg)
        else:
            box_thre, coef_thre, class_ids, class_thre = traditional_nms(box_thre, coef_thre, class_thre, cfg)

        return class_ids, class_thre, box_thre, coef_thre, proto_p


def nms_numpy(class_pred, box_pred, coef_pred, proto_out, anchors, cfg):
    class_p = class_pred.squeeze()  # [19248, 81]
    box_p = box_pred.squeeze()  # [19248, 4]
    coef_p = coef_pred.squeeze()  # [19248, 32]
    proto_p = proto_out.squeeze()  # [138, 138, 32]
    anchors = np.array(anchors).reshape(-1, 4)

    class_p = class_p.transpose(1, 0)
    # exclude the background class
    class_p = class_p[1:, :]
    # get the max score class of 19248 predicted boxes

    class_p_max = np.max(class_p, axis=0)  # [19248]

    # filter predicted boxes according the class score
    keep = (class_p_max > cfg.nms_score_thre)
    class_thre = class_p[:, keep]

    box_thre, anchor_thre, coef_thre = box_p[keep, :], anchors[keep, :], coef_p[keep, :]

    # decode boxes
    box_thre = np.concatenate((anchor_thre[:, :2] + box_thre[:, :2] * 0.1 * anchor_thre[:, 2:],
                               anchor_thre[:, 2:] * np.exp(box_thre[:, 2:] * 0.2)), axis=1)
    box_thre[:, :2] -= box_thre[:, 2:] / 2
    box_thre[:, 2:] += box_thre[:, :2]

    if class_thre.shape[1] == 0:
        return None, None, None, None, None
    else:
        assert not cfg.traditional_nms, 'Traditional nms is not supported with numpy.'
        box_thre, coef_thre, class_ids, class_thre = fast_nms_numpy(box_thre, coef_thre, class_thre, cfg)
        return class_ids, class_thre, box_thre, coef_thre, proto_p


def after_nms(ids_p, class_p, box_p, coef_p, proto_p, img_h, img_w, cfg=None, img_name=None):
    if ids_p is None:
        return None, None, None, None

    if cfg and cfg.visual_thre > 0:
        keep = class_p >= cfg.visual_thre
        if not keep.any():
            return None, None, None, None

        ids_p = ids_p[keep]
        class_p = class_p[keep]
        box_p = box_p[keep]
        coef_p = coef_p[keep]

    if cfg and cfg.save_lincomb:
        draw_lincomb(proto_p, coef_p, img_name)

    masks = torch.sigmoid(torch.matmul(proto_p, coef_p.t()))

    if not cfg or not cfg.no_crop:  # Crop masks by box_p
        masks = crop_box(masks, box_p)

    masks = masks.permute(2, 0, 1).contiguous()

    ori_size = max(img_h, img_w)
    # in OpenCV, cv2.resize is `align_corners=False`.
    masks = F.interpolate(masks.unsqueeze(0), (ori_size, ori_size), mode='bilinear', align_corners=False).squeeze(0)
    masks.gt_(0.5)  # Binarize the masks because of interpolation.
    masks = masks[:, 0: img_h, :] if img_h < img_w else masks[:, :, 0: img_w]

    box_p *= ori_size
    box_p = box_p.int()

    return ids_p, class_p, box_p, masks


def after_nms_numpy(ids_p, class_p, box_p, coef_p, proto_p, img_h, img_w, cfg=None):
    def np_sigmoid(x):
        return 1 / (1 + np.exp(-x))

    if ids_p is None:
        return None, None, None, None

    if cfg and cfg.visual_thre > 0:
        keep = class_p >= cfg.visual_thre
        if not keep.any():
            return None, None, None, None

        ids_p = ids_p[keep]
        class_p = class_p[keep]
        box_p = box_p[keep]
        coef_p = coef_p[keep]

    assert not cfg.save_lincomb, 'save_lincomb is not supported in onnx mode.'

    masks = np_sigmoid(np.matmul(proto_p, coef_p.T))

    if not cfg or not cfg.no_crop:  # Crop masks by box_p
        masks = crop_numpy(masks, box_p)

    ori_size = max(img_h, img_w)
    masks = cv2.resize(masks, (ori_size, ori_size), interpolation=cv2.INTER_LINEAR)

    if masks.ndim == 2:
        masks = masks[:, :, None]

    masks = np.transpose(masks, (2, 0, 1))
    masks = masks > 0.5  # Binarize the masks because of interpolation.
    masks = masks[:, 0: img_h, :] if img_h < img_w else masks[:, :, 0: img_w]

    box_p *= ori_size
    box_p = box_p.astype('int32')

    return ids_p, class_p, box_p, masks


def draw_lincomb(proto_data, masks, img_name):
    for kdx in range(1):
        jdx = kdx + 0
        coeffs = masks[jdx, :].cpu().numpy()
        idx = np.argsort(-np.abs(coeffs))

        coeffs_sort = coeffs[idx]
        arr_h, arr_w = (4, 8)
        p_h, p_w, _ = proto_data.size()
        arr_img = np.zeros([p_h * arr_h, p_w * arr_w])
        arr_run = np.zeros([p_h * arr_h, p_w * arr_w])

        for y in range(arr_h):
            for x in range(arr_w):
                i = arr_w * y + x

                if i == 0:
                    running_total = proto_data[:, :, idx[i]].cpu().numpy() * coeffs_sort[i]
                else:
                    running_total += proto_data[:, :, idx[i]].cpu().numpy() * coeffs_sort[i]

                running_total_nonlin = (1 / (1 + np.exp(-running_total)))

                arr_img[y * p_h:(y + 1) * p_h, x * p_w:(x + 1) * p_w] = (proto_data[:, :, idx[i]] / torch.max(
                    proto_data[:, :, idx[i]])).cpu().numpy() * coeffs_sort[i]
                arr_run[y * p_h:(y + 1) * p_h, x * p_w:(x + 1) * p_w] = (running_total_nonlin > 0.5).astype(np.float)

        arr_img = ((arr_img + 1) * 127.5).astype('uint8')
        arr_img = cv2.applyColorMap(arr_img, cv2.COLORMAP_WINTER)
        cv2.imwrite(f'/content/drive/MyDrive/yolact/results/images/lincomb_{img_name}', arr_img)


def draw_img(ids_p, class_p, box_p, mask_p, img_origin, cfg, img_name=None, fps=None):
    if ids_p is None:
        return img_origin

    if isinstance(ids_p, torch.Tensor):
        ids_p = ids_p.cpu().numpy()
        class_p = class_p.cpu().numpy()
        box_p = box_p.cpu().numpy()
        mask_p = mask_p.cpu().numpy()

    num_detected = ids_p.shape[0]

    img_fused = img_origin
    if not cfg.hide_mask:
        masks_semantic = mask_p * (ids_p[:, None, None] + 1)  # expand ids_p' shape for broadcasting
        # The color of the overlap area is different because of the '%' operation.
        masks_semantic = masks_semantic.astype('int').sum(axis=0) % (cfg.num_classes - 1)
        color_masks = COLORS[masks_semantic].astype('uint8')
        img_fused = cv2.addWeighted(color_masks, 0.4, img_origin, 0.6, gamma=0)

        if cfg.cutout:
            total_obj = (masks_semantic != 0)[:, :, None].repeat(3, 2)
            total_obj = total_obj * img_origin
            new_mask = ((masks_semantic == 0) * 255)[:, :, None].repeat(3, 2)
            img_matting = (total_obj + new_mask).astype('uint8')
            cv2.imwrite(f'/content/drive/MyDrive/yolact/results/images/{img_name}_total_obj.jpg', img_matting)

            for i in range(num_detected):
                one_obj = (mask_p[i])[:, :, None].repeat(3, 2)
                one_obj = one_obj * img_origin
                new_mask = ((mask_p[i] == 0) * 255)[:, :, None].repeat(3, 2)
                x1, y1, x2, y2 = box_p[i, :]
                img_matting = (one_obj + new_mask)[y1:y2, x1:x2, :]
                cv2.imwrite(f'/content/drive/MyDrive/yolact/results/images/{img_name}_{i}.jpg', img_matting)
    scale = 0.6
    thickness = 1
    font = cv2.FONT_HERSHEY_DUPLEX

    if not cfg.hide_bbox:
        for i in reversed(range(num_detected)):
            x1, y1, x2, y2 = box_p[i, :]

            color = COLORS[ids_p[i] + 1].tolist()
            cv2.rectangle(img_fused, (x1, y1), (x2, y2), color, thickness)

            class_name = cfg.class_names[ids_p[i]]
            text_str = f'{class_name}: {class_p[i]:.2f}' if not cfg.hide_score else class_name

            text_w, text_h = cv2.getTextSize(text_str, font, scale, thickness)[0]
            cv2.rectangle(img_fused, (x1, y1), (x1 + text_w, y1 + text_h + 5), color, -1)
            cv2.putText(img_fused, text_str, (x1, y1 + 15), font, scale, (255, 255, 255), thickness, cv2.LINE_AA)

    if cfg.real_time:
        fps_str = f'fps: {fps:.2f}'
        text_w, text_h = cv2.getTextSize(fps_str, font, scale, thickness)[0]
        # Create a shadow to show the fps more clearly
        img_fused = img_fused.astype(np.float32)
        img_fused[0:text_h + 8, 0:text_w + 8] *= 0.6
        img_fused = img_fused.astype(np.uint8)
        cv2.putText(img_fused, fps_str, (0, text_h + 2), font, scale, (255, 255, 255), thickness, cv2.LINE_AA)

    return img_fused


In [11]:
import time
import torch
import numpy as np

times = {}
times.setdefault('batch', [])
times.setdefault('data', [])
mark = False  # Use for starting and stopping the timer
max_len = 100
cuda = torch.cuda.is_available()


def reset(length=100):
    global times, mark, max_len
    times = {}
    times.setdefault('batch', [])
    times.setdefault('data', [])
    mark = False
    max_len = length


def start():
    global mark, times
    mark = True

    for k, v in times.items():
        if len(v) != 0:
            print('Warning, time list is not empty when starting.')


def add_batch_time(batch_time):
    if mark:
        times['batch'].append(batch_time)

        inner_time = 0
        for k, v in times.items():
            if k not in ('batch', 'data'):
                inner_time += v[-1]

        times['data'].append(batch_time - inner_time)


def get_times(time_name):
    return_time = []
    for name in time_name:
        return_time.append(np.mean(times[name]))

    return return_time


class counter:
    def __init__(self, name, trt_mode=False):
        self.name = name
        self.times = times
        self.mark = mark
        self.max_len = max_len
        self.trt_mode = trt_mode

        for v in times.values():
            if len(v) >= self.max_len:  # pop the first item if the list is full
                v.pop(0)

    def __enter__(self):
        if self.mark:
            if cuda and (not self.trt_mode):  # cuda operation in torch should be forbidden when run by TensorRT
                torch.cuda.synchronize()

            self.times.setdefault(self.name, [])
            self.times[self.name].append(time.perf_counter())

    def __exit__(self, e, ev, t):
        if self.mark:
            if cuda and (not self.trt_mode):
                torch.cuda.synchronize()

            self.times[self.name][-1] = time.perf_counter() - self.times[self.name][-1]


In [12]:
import torch
import torch.nn as nn


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, norm_layer=nn.BatchNorm2d):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = norm_layer(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = norm_layer(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = norm_layer(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    """ Adapted from torchvision.models.resnet """

    def __init__(self, layers, block=Bottleneck, norm_layer=nn.BatchNorm2d):
        super().__init__()

        self.num_base_layers = len(layers)
        self.layers = nn.ModuleList()
        self.channels = []
        self.norm_layer = norm_layer
        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self._make_layer(block, 64, layers[0])
        self._make_layer(block, 128, layers[1], stride=2)
        self._make_layer(block, 256, layers[2], stride=2)
        self._make_layer(block, 512, layers[3], stride=2)

        self.backbone_modules = [m for m in self.modules() if isinstance(m, nn.Conv2d)]

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None

        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(nn.Conv2d(self.inplanes, planes * block.expansion,
                                                 kernel_size=1, stride=stride, bias=False),
                                       self.norm_layer(planes * block.expansion))

        layers = [block(self.inplanes, planes, stride, downsample, self.norm_layer)]
        self.inplanes = planes * block.expansion

        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, norm_layer=self.norm_layer))

        layer = nn.Sequential(*layers)

        self.channels.append(planes * block.expansion)
        self.layers.append(layer)

    def forward(self, x):
        """ Returns a list of convouts for each layer. """
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        outs = []
        for i, layer in enumerate(self.layers):
            x = layer(x)
            outs.append(x)

        return tuple(outs)

    def init_backbone(self, path):
        """ Initializes the backbone weights for training. """
        state_dict = torch.load(path)
        self.load_state_dict(state_dict, strict=True)
        print(f'\nBackbone is initiated with {path}.\n')


In [13]:
import types

args= types.SimpleNamespace()

args.klas = 'res101_coco'
args.train_bs = 8
args.img_size = 544
args.resume = None
args.val_interval = 4000
args.img_size = 544
args.weight = None
args.traditional_nms = False
args.val_num = -1
args.coco_api = False

# cfg = types.SimpleNamespace()

In [14]:
import re
import torch
import time
import torch.utils.data as data
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torch.backends.cudnn as cudnn



iou_thres = [x / 100 for x in range(50, 100, 5)]
make_json = MakeJson()


def evaluate(net, cfg, step=None):
    dataset = COCODetection(cfg, mode='val')
    data_loader = data.DataLoader(dataset, 1, num_workers=4, shuffle=False, pin_memory=True, collate_fn=val_collate)
    ds = len(data_loader)
    progress_bar = ProgressBar(40, ds)
    reset()

    ap_data = {'box': [[APDataObject() for _ in cfg.class_names] for _ in iou_thres],
               'mask': [[APDataObject() for _ in cfg.class_names] for _ in iou_thres]}

    for i, (img, gt, gt_masks, img_h, img_w) in enumerate(data_loader):
        if i == 1:
            start()

        if cfg.cuda:
            img, gt, gt_masks = img.cuda(), gt.cuda(), gt_masks.cuda()

        with torch.no_grad(), counter('forward'):
            class_p, box_p, coef_p, proto_p = net(img)

        with counter('nms'):
            ids_p, class_p, box_p, coef_p, proto_p = nms(class_p, box_p, coef_p, proto_p, net.anchors, cfg)

        with counter('after_nms'):
            ids_p, class_p, boxes_p, masks_p = after_nms(ids_p, class_p, box_p, coef_p, proto_p, img_h, img_w)
            if ids_p is None:
                continue

        with counter('metric'):
            ids_p = list(ids_p.cpu().numpy().astype(int))
            class_p = list(class_p.cpu().numpy().astype(float))

            if cfg.coco_api:
                boxes_p = boxes_p.cpu().numpy()
                masks_p = masks_p.cpu().numpy()

                for j in range(masks_p.shape[0]):
                    if (boxes_p[j, 3] - boxes_p[j, 1]) * (boxes_p[j, 2] - boxes_p[j, 0]) > 0:
                        make_json.add_bbox(dataset.ids[i], ids_p[j], boxes_p[j, :], class_p[j])
                        make_json.add_mask(dataset.ids[i], ids_p[j], masks_p[j, :, :], class_p[j])
            else:
                prep_metrics(ap_data, ids_p, class_p, boxes_p, masks_p, gt, gt_masks, img_h, img_w, iou_thres)

        aa = time.perf_counter()
        if i > 0:
            batch_time = aa - temp
            add_batch_time(batch_time)
        temp = aa

        if i > 0:
            t_t, t_d, t_f, t_nms, t_an, t_me = get_times(['batch', 'data', 'forward',
                                                                'nms', 'after_nms', 'metric'])
            fps, t_fps = 1 / (t_d + t_f + t_nms + t_an), 1 / t_t
            bar_str = progress_bar.get_bar(i + 1)
            print(f'\rTesting: {bar_str} {i + 1}/{ds}, fps: {fps:.2f} | total fps: {t_fps:.2f} | '
                  f't_t: {t_t:.3f} | t_d: {t_d:.3f} | t_f: {t_f:.3f} | t_nms: {t_nms:.3f} | '
                  f't_after_nms: {t_an:.3f} | t_metric: {t_me:.3f}', end='')

    table, box_row, mask_row = calc_map(ap_data, iou_thres, len(cfg.class_names), step=step)
    print(table)
    return table, box_row, mask_row


if __name__ == '__main__':


    # Load or initialize configuration
    # prefix = re.findall(r'best_\d+\.\d+_', args.weight)[0] if args.weight else 'best_'
    # suffix = re.findall(r'_\d+\.pth', args.weight)[0] if args.weight else '.pth'
    # args.klas = args.weight.split(prefix)[-1].split(suffix)[0] if args.weight else 'res101_coco'
    args.klas = 'res101_coco'
    cfg = get_config(args, mode='val')  # Assuming get_config initializes training configuration

    net = Yolact(cfg)

    if not args.weight:
        # net.apply(init_weights)  # Assuming init_weights is a function that initializes the model weights
        net.init_weights(net)
        # init_weights(net)
        # net.init_weights()
    else:
        net.load_weights(cfg.weight, cfg.cuda)
    net.eval()

    if cfg.cuda:
        cudnn.benchmark = True
        cudnn.fastest = True
        net = net.cuda()

    evaluate(net, cfg)



-----No GPU found, validate on CPU.-----

------------------------------res101_coco------------------------------
mode: val
cuda: False
gpu_id: 0
img_size: 544
class_names: ('cells', 'kernel')
num_classes: 3
scales: [24, 48, 96, 192, 384]
aspect_ratios: [1, 0.5, 2]
weight: None
val_imgs: /content/drive/MyDrive/custom_dataset/valid/
val_ann: /content/drive/MyDrive/custom_dataset/valid/_annotations.coco.json
val_bs: 1
val_num: -1
coco_api: False
traditional_nms: False
nms_score_thre: 0.05
nms_iou_thre: 0.5
top_k: 200
max_detections: 200

Model initialized.

loading annotations into memory...
Done (t=1.99s)
creating index...
index created!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Testing: ████████████████████████████████████████ 123/123, fps: 0.24 | total fps: 0.17 | t_t: 5.846 | t_d: 0.050 | t_f: 3.201 | t_nms: 0.029 | t_after_nms: 0.888 | t_metric: 1.683
Calculating mAP...
+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|      | all | 50  | 55  | 60  | 65  | 70  | 75  | 80  | 85  | 90  | 95  |
+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| box  | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
| mask | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |
+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+


In [18]:
# !pip install tensorboardX
import time
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter
# import torch.utils.data as data
from torch.utils.data import DataLoader
import datetime
import re

cfg = get_config(args, mode='train')
cfg_name = cfg.__class__.__name__

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.is_available():
#     num_gpu = torch.cuda.device_count()
#     print('num of GPU: {num_gpu}')
#     main_gpu = 0
#     torch.cuda.set_device(main_gpu)
#     device = torch.device("cuda", main_gpu)
# else:
#     device = torch.device("cpu")

net = Yolact(cfg)
net.to(device)

# net.train()

# if not args.weight:
#         net.apply(init_weights)

# if args.resume:
#     assert re.findall(r'res.+_[a-z]+', args.resume)[0] == cfg_name, 'Resume weight is not compatible with current cfg.'
#     net.load_weights(cfg.weight, cfg.cuda)
#     start_step = int(cfg.weight.split('.pth')[0].split('_')[-1])
# else:
#     net.backbone.init_backbone(cfg.weight)
#     start_step = 0

optimizer = optim.SGD(net.parameters(), lr=cfg.lr, momentum=0.9, weight_decay=5e-4)
num_epochs = 600

dataset = COCODetection(cfg, mode='train')

# train_sampler = None
if cfg.cuda:
    cudnn.benchmark = True
    cudnn.fastest = True

# shuffle must be False if sampler is specified
data_loader = DataLoader(dataset, args.train_bs, shuffle=True, pin_memory=False)
# data_loader = data.DataLoader(dataset, cfg.bs_per_gpu, num_workers=0, shuffle=False,
#                               collate_fn=train_collate, pin_memory=True) sampler=train_sampler)
start_step = 0
epoch_seed = 0
map_tables = []
# training = True
reset()
step = start_step
val_step = start_step
writer = SummaryWriter(f'/content/drive/MyDrive/yolact/tensorboard_log/{cfg_name}')


try:  # try-except can shut down all processes after Ctrl + C.
    # while training:
    for epoch in range(num_epochs):
        net.train()
        train_loss = 0.0
        # if train_sampler:
        #     epoch_seed += 1
        #     train_sampler.set_epoch(epoch_seed)

        for images, targets, masks in data_loader:
            if not cfg.cuda:
              images = images.to(device)
              targets = targets.to(device)
              masks = masks.to(device)
            if cfg.warmup_until > 0 and step <= cfg.warmup_until:  # warm up learning rate.
                for param_group in optimizer.param_groups:
                    param_group['lr'] = (cfg.lr - cfg.warmup_init) * (step / cfg.warmup_until) + cfg.warmup_init

            if step in cfg.lr_steps:  # learning rate decay.
                for param_group in optimizer.param_groups:
                    param_group['lr'] = cfg.lr * 0.1 ** cfg.lr_steps.index(step)

            if cfg.cuda:
                images = images.cuda().detach()
                targets = [ann.cuda().detach() for ann in targets]
                masks = [mask.cuda().detach() for mask in masks]

            with counter('for+loss'):
                loss_c, loss_b, loss_m, loss_s = net(images, targets, masks)

                if cfg.cuda:
                    all_losses = [loss_c, loss_b, loss_m, loss_s]
                    all_loss = sum(all_losses)


            with counter('backward'):
                loss_total = loss_c + loss_b + loss_m + loss_s
                optimizer.zero_grad()
                loss_total.backward()

            with counter('update'):
                optimizer.step()

            time_this = time.time()
            if step > start_step:
                batch_time = time_this - time_last
                add_batch_time(batch_time)
            time_last = time_this

            if step % 10 == 0 and step != start_step:
                if (not cfg.cuda):
                    cur_lr = optimizer.param_groups[0]['lr']
                    time_name = ['batch', 'data', 'for+loss', 'backward', 'update']
                    t_t, t_d, t_fl, t_b, t_u = get_times(time_name)
                    seconds = (cfg.lr_steps[-1] - step) * t_t
                    eta = str(datetime.timedelta(seconds=seconds)).split('.')[0]

                    # Get the mean loss across all GPUS for printing, seems need to call .item(), not sure
                    # l_c = all_loss[0].item() / num_gpu if main_gpu else loss_c.item()
                    # l_b = all_loss[1].item() / num_gpu if main_gpu else loss_b.item()
                    # l_m = all_loss[2].item() / num_gpu if main_gpu else loss_m.item()
                    # l_s = all_loss[3].item() / num_gpu if main_gpu else loss_s.item()
                    l_c = loss_c.item()
                    l_b = loss_b.item()
                    l_m = loss_m.item()
                    l_s = loss_s.item()

                    writer.add_scalar('loss/class', l_c, global_step=step)
                    writer.add_scalar('loss/box', l_b, global_step=step)
                    writer.add_scalar('loss/mask', l_m, global_step=step)
                    writer.add_scalar('loss/semantic', l_s, global_step=step)
                    writer.add_scalar('loss/total', loss_total, global_step=step)

                    print(f'step: {step} | lr: {cur_lr:.2e} | l_class: {l_c:.3f} | l_box: {l_b:.3f} | '
                          f'l_mask: {l_m:.3f} | l_semantic: {l_s:.3f} | t_t: {t_t:.3f} | t_d: {t_d:.3f} | '
                          f't_fl: {t_fl:.3f} | t_b: {t_b:.3f} | t_u: {t_u:.3f} | ETA: {eta}')

            if args.val_interval > 0 and step % args.val_interval == 0 and step != start_step:
                if (not cfg.cuda):
                    val_step = step
                    net.eval()
                    table, box_row, mask_row = evaluate(net.module, cfg, step)
                    map_tables.append(table)
                    net.train()
                    reset()  # training timer and val timer share the same Obj, so reset it to avoid conflict

                    writer.add_scalar('mAP/box_map', box_row[1], global_step=step)
                    writer.add_scalar('mAP/mask_map', mask_row[1], global_step=step)

                    save_best(net.module if cfg.cuda else net, mask_row[1], cfg_name, step)

            if (not cfg.cuda) and step == val_step + 1:
                start()  # the first iteration after validation should not be included

            step += 1
            if step >= cfg.lr_steps[-1]:
                training = False

                if not cfg.cuda:
                    save_latest(net.module if cfg.cuda else net, cfg_name, step)

                    print('\nValidation results during training:\n')
                    for table in map_tables:
                        print(table, '\n')

                    print(f'Training completed.')

                break

except KeyboardInterrupt:
    if (not cfg.cuda):
        save_latest(net.module if cfg.cuda else net, cfg_name, step)

        print('\nValidation results during training:\n')
        for table in map_tables:
            print(table, '\n')






-----No GPU found, training on CPU.-----

------------------------------res101_coco------------------------------
mode: train
cuda: False
gpu_id: 0
img_size: 544
class_names: ('cells', 'kernel')
num_classes: 3
scales: [24, 48, 96, 192, 384]
aspect_ratios: [1, 0.5, 2]
weight: /content/drive/MyDrive/yolact/weights/backbone_res101.pth
train_imgs: /content/drive/MyDrive/custom_dataset/train/
train_ann: /content/drive/MyDrive/custom_dataset/train/_annotations.coco.json
train_bs: 8
bs_per_gpu: 8
val_interval: 4000
bs_factor: 1.0
lr: 0.001
warmup_init: 0.0001
warmup_until: 500
lr_steps: (0, 280000, 560000, 620000, 680000)
pos_iou_thre: 0.5
neg_iou_thre: 0.4
conf_alpha: 1
bbox_alpha: 1.5
mask_alpha: 6.125
semantic_alpha: 1
masks_to_train: 200
val_imgs: /content/drive/MyDrive/custom_dataset/valid/
val_ann: /content/drive/MyDrive/custom_dataset/valid/_annotations.coco.json
val_bs: 1
val_num: -1
coco_api: False
traditional_nms: False
nms_score_thre: 0.05
nms_iou_thre: 0.5
top_k: 200
max_detectio

<ipython-input-4-50f6aa71f90f>:116: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  img_cropped = img[int(new_y1[0]): int(new_y2[0]), int(new_x1[0]): int(new_x2[0]), :]
<ipython-input-4-50f6aa71f90f>:117: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  masks_remained = masks_remained[:, int(new_y1[0]): int(new_y2[0]), int(new_x1[0]): int(new_x2[0])]


TypeError: cannot unpack non-iterable NoneType object